### 01 - Problem (case study)

.- El objetivo de estos datos es comprender la demografía de los clientes y el comportamiento de compra. Ademas se debe comprobar los clientes mas rentables y como interactuan. 

.- La descripción de datos se realiza acontinuación:

In [ ]:
#Importamos todas las librerias necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import norm
from scipy import stats 
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.formula.api import ols

### 02 - Getting Data

In [ ]:
#importamos el CSV como un Dataframe llamado DF
df=pd.read_csv("marketing_customer_analysis.csv")

### 03 - Cleaning/Wrangling/EDA

In [ ]:
#en este paso se tiene que hacer toda la limpieza del Data Frame, completar los Nan values, mirar los tipos de variables etc. 
#estandarizo los nombres de columnas y realizo una copia.

df.columns=df.columns.str.upper().str.replace(' ','_')
df1=df.copy()

In [ ]:
# Hago la distinción de variables numericas y categoricas. 

numerical = df1.select_dtypes(include=np.number)
categoricals = df1.select_dtypes(include=object)

### 04 - Processing Data

In [ ]:
#Realizo graficos de control para poder comprobar la distribución de mis variables. 

fig, axes = plt.subplots(len(numerical.columns), 1, figsize=(4, 2* len(numerical.columns)))

for i, column in enumerate(numerical.columns):
    sns.boxplot(numerical[column], ax=axes[i])
    axes[i].set_title(column)

plt.tight_layout()
plt.show()  

In [ ]:
# VARIABLES NUMERICAS. 

# Normalizo la variables numericas. 
transformer=StandardScaler().fit(numerical)
x_normalized=transformer.transform(numerical)
df_norm=pd.DataFrame(x_normalized)

#Cambio nombres del Df normalizado y lo reemplazo por los nombres d elas columnas del DF numerico.
col_num=numerical.columns
df_norm.columns=col_num
df_norm

In [ ]:
# VARIABLES CATEGORICAS. 

#Elimino columnas que no tienen implicación en el analisis (aquellas columnas que tienen muchos elementos que se repiten solo una vez y no tienen relevancia para el modelo. al no eliminarlas, cuando se vaya a hacer el hot encoder, se generaran muchas columnas que alteraran el tamaño del dataframe). 

#Defino un nuevo DF sin esas columnas. 

df_categ=categoricals.drop(['EFFECTIVE_TO_DATE', 'CUSTOMER'],axis=1)
df_categ

#aplico el hot encoder a las variables numericas. 

encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(df_categ)
encoded = encoder.transform(df_categ).toarray()
feature_names = encoder.get_feature_names_out(df_categ.columns)
X_cat_encoded = pd.DataFrame(encoded, columns=feature_names)
df_encod = df_categ.drop(df_categ.columns, axis=1)
df_encod = pd.concat([df_encod, X_cat_encoded], axis=1)
df_encod

In [72]:
#concateno ambos DF (categorico y numerico) ya trabajados. 

df_concat=pd.concat([df_norm, df_encod], axis=1 )
df_concat=pd.DataFrame(df_concat)

In [ ]:
#Realizo un train test con una muestra de 20%.

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score

Y=df_concat['TOTAL_CLAIM_AMOUNT']
X=df_concat.drop(['TOTAL_CLAIM_AMOUNT'] , axis=1)
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, random_state=42)
lm=linear_model.LinearRegression()
model=lm.fit(X_train, y_train)
predictions=lm.predict(X_test)
r2_score(y_test, predictions)


In [ ]:
# Se transforma el resultado del Train test a Dataframe, solo para control. 
dfpred=pd.DataFrame(columns=[y_test, predictions]).T
dfpred

### 05 - Modeling

In [ ]:
#Realizo la 1ª regresión lineal, sobre el Df concatenado.
Y=df_concat['TOTAL_CLAIM_AMOUNT']
X=df_concat.drop(['TOTAL_CLAIM_AMOUNT'], axis=1)
x=sm.add_constant(X)
model=sm.OLS(Y,X).fit()
print(model.summary())

In [ ]:
#Al encontrarme con variables con un P valor alto, pruebo eliminarlas, para ver si el modelo mejora.
df2_concat=df_concat.copy()
df2_concat=df_concat.drop(['MONTHS_SINCE_POLICY_INCEPTION'] , axis=1)

In [ ]:
#Realizo una 2ª regresion lineal sin la columna MONTHS_SINCE_POLICY_INCEPTION 
Y=df2_concat['TOTAL_CLAIM_AMOUNT']
X=df2_concat.drop(['TOTAL_CLAIM_AMOUNT'], axis=1)
x=sm.add_constant(X)
model=sm.OLS(Y,X).fit()
print(model.summary())

In [57]:
#al ver que la regresion lineal no ha cambiado quitando 1 columna, mirare las que tengan mas P valor. 

Y = df_concat['TOTAL_CLAIM_AMOUNT']
X = df_concat.drop(['TOTAL_CLAIM_AMOUNT'], axis=1)
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

# Filtrar variables con el p-valor más alto (por ejemplo, umbral de 0.05)

significant_variables = model.pvalues[model.pvalues > 0.05]
significant_variables_sorted = significant_variables.sort_values(ascending=False)
significant_variables_sorted


In [ ]:
#Hago una ultima prueba, quitaré las columnas con variables con P valor alta y que no sean relevantes para el modelo.
df3_concat=df2_concat.copy()
df3_concat=df2_concat.drop(['VEHICLE_CLASS_Two-Door Car', 'VEHICLE_CLASS_Luxury SUV', 'POLICY_Corporate L3', 'VEHICLE_SIZE_Small', 'SALES_CHANNEL_Web', 'POLICY_TYPE_Personal Auto', 'EMPLOYMENTSTATUS_Retired', 'POLICY_Corporate L2', 'SALES_CHANNEL_Branch', ] , axis=1)

In [ ]:
#Ralizo una 3ª regresion lineal despues de haber eliminado unas cuantas columnas. 

Y = df3_concat['TOTAL_CLAIM_AMOUNT']
X = df3_concat.drop(['TOTAL_CLAIM_AMOUNT'], axis=1)
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

### 06 - Model Validation

In [ ]:
#varianza
mse=mean_squared_error(y_test, predictions) 
print("MSE es =" , mse)

#error estándar
rmse= math.sqrt(mse) 
print("RMSE es =", rmse)

# valores de 0 a 1, cuanto más cercano a 1 mejor. 
r2=r2_score(y_test, predictions) 
print("R2 es =", r2)

#igual que r2 pero penaliza el incluir variables independientes que no aporten al modelo.
n=len(X_test)  
p=X_test.shape[1]
adj_r2=1-((1-r2)*(n-1)/(n-p-1))
print("R2 AJUSTADO es =", adj_r2)

### 07 - Reporting

Informe de Regresión Lineal

se analizó un modelo de regresión lineal para predecir el "TOTAL CLAIM AMOUNT" (Monto total de reclamaciones). 

El modelo resultante mostró un coeficiente de determinación (R2) de 0.761, lo que indica que aproximadamente el 76.1% de la variabilidad en el monto total de reclamaciones se explica por las variables independientes incluidas en el modelo. Además, el valor ajustado del R2 fue de 0.760, lo que sugiere que el modelo también se ajusta adecuadamente a nuevos datos.

El análisis de los coeficientes estimados reveló las siguientes relaciones significativas con el "TOTAL CLAIM AMOUNT":

La variable "MONTHLY PREMIUM AUTO" mostró un coeficiente positivo de 0.8318, aqui se podria inferir que hay una vinculación entre el tipo de poliza y el monto de las reclamaciones.

Las variables "STATE_California", "STATE_Nevada", "STATE_Oregon" y "STATE_Washington" mostraron coeficientes negativos, lo que sugiere una relación inversa entre variables. 

El análisis de los p-valores reveló muchas de las variables del modelo tenian un valor superior a 0.05.

Con la ultima (3ª) regresión, se puede comprobar que al eliminar varias columnas con P valor alto (no todas) el R2 y el R2 ajustado han aumentado, por lo que se puede concluir que el modelo esta mas ajustado a la realidad. De igual forma la primera regresión tenía un R2 de 0,761, el cual es un resultado "aceptable". Otro aspecto a tener en cuenta, es que hay un problema de multicolinealidad, lo que podria llegar a dificultar las interpretaciones del modelo. No es conveniente eliminar todas las variables que consideremos no utiles, ya que se debe encontrar un equilibrio entre reducir la multicolinealidad y mantener la capacidad explicativa del modelo.
